In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import numpy as np 
import pandas as pd

In [3]:
dataset_path = "./dataset.csv"

df = pd.read_csv(
    dataset_path,
    # dtype is specified to prevent a DtypeWarning
    dtype={
        "full_query": str,
        "label": int,
        "statement_type": str,
        "query_template_id": str,
        "attack_payload": str,
        "attack_id": str,
        "attack_technique": str,
        "attack_desc": str,
        "split": str,
        "sqlmap_status" : str,
        "attack_stage" : str,
        "tamper_method" : str

    },
)

display(df)


,full_query,label,attack_payload,attack_desc,attack_stage,tamper_method,sqlmap_status,statement_type,query_template_id,attack_id,attack_technique,split
0,"DELETE FROM navaids WHERE ident = ""PO"";",0,NaN,NaN,NaN,NaN,NaN,delete,airport-D6,NaN,NaN,test
1,"SELECT a.id, a.name, a.municipality, a.iso_cou...",1,NaN,NaN,recon,commentbeforeparentheses,NaN,select,airport-S10,error-3,error,test
2,"SELECT a.id, a.name, a.municipality, a.iso_cou...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S10,NaN,NaN,test
3,"INSERT INTO regions (code, name, continent, is...",1,NaN,NaN,recon,space2mysqlblank,NaN,insert,airport-I4,error-9,error,test
4,"SELECT a.id, a.name, a.municipality, a.iso_cou...",1,NaN,NaN,recon,randomcase,NaN,select,airport-S10,error-3,error,test
...,...,...,...,...,...,...,...,...,...,...,...,...
37011,"SELECT a.id, a.ident, a.name, a.latitude_deg, ...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S14,NaN,NaN,test
37012,"INSERT INTO regions (code, name, continent, is...",0,NaN,NaN,NaN,NaN,NaN,insert,airport-I4,NaN,NaN,test
37013,"SELECT a.id, a.ident, a.name, a.latitude_deg, ...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S14,NaN,NaN,test
37014,"SELECT a.id, a.ident, a.name, a.latitude_deg, ...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S14,NaN,NaN,test


In [4]:
def display_attack_normal_per_set(df : pd.DataFrame):
    df_train_atk = df[(df["label"] == 1) & (df["split"] == "train")]  
    df_train_normal = df[(df["label"] == 0) & (df["split"] == "train")]  

    df_test_atk = df[(df["label"] == 1) & (df["split"] == "test")]  
    df_test_normal = df[(df["label"] == 0) & (df["split"] == "test")]  

    print(f"Number of attacks in train: {df_train_atk.shape[0]}")
    print(f"Number of normal in train: {df_train_normal.shape[0]}")
    print(f"Number of attacks in test: {df_test_atk.shape[0]}")
    print(f"Number of normal in test: {df_test_normal.shape[0]}")

display_attack_normal_per_set(df)

Number of attacks in train: 4132
Number of normal in train: 4261
Number of attacks in test: 2382
Number of normal in test: 26241


In [10]:
def display_ids_per_set(df: pd.DataFrame):
    df_train = df[df["split"] == "train"]
    df_test = df[df["split"] == "test"]

    tids_train = df_train["query_template_id"].unique()
    tids_test = df_test["query_template_id"].unique()
    tids_complement = set(tids_test) - set(tids_train)
    print(f"Template IDS in df train: {len(tids_train)}, {tids_train}")
    print(f"Template IDS in df test:   {len(tids_test)}, {tids_test}")
    print(
        f"Template IDs in train but NOT in test: {len(tids_complement)}, {sorted(tids_complement)}"
    )

    # Compute which template only have normal queries
    df_n = df.loc[df["label"] == 0,"query_template_id"].unique()
    df_a = df.loc[df["label"] == 1,"query_template_id"].unique()

    print(f"Templates IDs with no attacks: {set(df_n) - set(df_a)}")




display_ids_per_set(df)

Template IDS in df train: 8, ['airport-I5' 'airport-S10' 'airport-D7' 'airport-S8' 'airport-D6'
 'airport-I9' 'airport-S11' 'airport-U3']
Template IDS in df test:   10, ['airport-D6' 'airport-S10' 'airport-I4' 'airport-I9' 'airport-I5'
 'airport-S11' 'airport-U3' 'airport-S8' 'airport-D7' 'airport-S14']
Template IDs in train but NOT in test: 2, ['airport-I4', 'airport-S14']
Templates IDs with no attacks: {'airport-D7'}
